<a href="https://colab.research.google.com/github/carloosarthuur/Atividade-2/blob/main/Letra_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import time
import pandas as pd

# Polinômio do Exemplo 1
# P(x) = x^5 - 3.7x^4 + 7.4x^3 - 10.8x^2 + 10.8x - 6.8
# Coeficientes do maior grau para o menor:
coefs_poly = [1, -3.7, 7.4, -10.8, 10.8, -6.8]

## Método de Newton para Polinômios

A principal característica dessa implementação é o uso do **Dispositivo de Briot-Ruffini**. Diferentemente da abordagem genérica, não há cálculo explícito de potências do polinômio.

Em um único laço `for`, são calculados simultaneamente:
1. O valor do polinômio no ponto, armazenado em `b`, isto é, $P(x)$.
2. O valor da derivada no ponto, acumulado em `c$, isto é, $P'(x)$.

A atualização da aproximação segue diretamente a fórmula do método de Newton:
$$
x_{k+1} = x_k - \frac{b}{c}.
$$


In [13]:
def newton_poly(coefs, x0, eps1, eps2, max_iter=100): # Dados iniciais
    start = time.perf_counter()
    n = len(coefs) - 1
    x = x0

    # Verificação rápida do chute inicial
    b = coefs[0]
    for i in range(1, n + 1):
        b = coefs[i] + b * x

    # Teste inicial
    if abs(b) < eps1:
        end = time.perf_counter()
        return {
            "Método": "Newton Poly",
            "Dados Iniciais": f"x0={x0}",
            "Raiz": x, "f(x)": b, "Erro": 0,
            "Iterações": 0, "Tempo Total": end - start, "Tempo/Iter": 0
        }

    k = 1 # inicia contador
    while k <= max_iter:
        # aplicando o Briot-Ruffini para calcular b e c
        b = coefs[0]
        c = b

        # calculando c
        for i in range(1, n):
            b = coefs[i] + b * x
            c = b + c * x  # c acumula o valor da derivada P'(x)

        # calculando b
        b = coefs[n] + b * x

        # Critério de parada pelo valor da função
        if abs(b) < eps1:
            end = time.perf_counter()
            t_total = end - start
            return {
                "Método": "Newton Poly", "Dados Iniciais": f"x0={x0}",
                "Raiz": x, "f(x)": b, "Erro": 0,
                "Iterações": k, "Tempo Total": t_total, "Tempo/Iter": t_total/k
            }

        if c == 0: break # impede divisão por 0

        # Atualização de Newton
        deltax = b / c
        x_new = x - deltax

        # Critério de parada pelo tamanho do passo
        if abs(deltax) < eps2:
            end = time.perf_counter()
            t_total = end - start
            return {
                "Método": "Newton Poly", "Dados Iniciais": f"x0={x0}",
                "Raiz": x_new, "f(x)": b, "Erro": abs(deltax),
                "Iterações": k, "Tempo Total": t_total, "Tempo/Iter": t_total/k
            }

        # Atualiza x
        x = x_new

        # Incrementa contador
        k = k + 1

    return None

## Método do Ponto Fixo – MPF (Adaptação)

Para adaptar a implementação ao Método do Ponto Fixo, foram realizadas as seguintes modificações:

1. **Remoção da derivada (`c`):** o MPF não utiliza derivadas. Assim, a variável `c`, presente no algoritmo de Briot-Ruffini para o método de Newton, foi removida.

2. **Definição da função de iteração ($\phi(x)$):** diferentemente do método de Newton, cuja atualização é genérica, o MPF exige a definição explícita de uma função de iteração $\phi(x)$ obtida a partir da equação polinomial $P(x)=0$.  
   Para o polinômio do **Exemplo 1**
   $$
   x^5 - 3.7x^4 + 7.4x^3 - 10.8x^2 + 10.8x - 6.8 = 0,
   $$
   isolou-se o termo linear $10.8x$, resultando na função implementada `phi(val_x)`.

3. **Papel da variável `b`:** o cálculo de `b` via método de Horner foi mantido apenas para a verificação do critério de parada, isto é,
   $$
   |f(x)| < \varepsilon_1.
   $$
   A variável `b` não participa da atualização do próximo ponto, que é realizada exclusivamente por meio da chamada `x1 = phi(x)`.


In [14]:
def mpf_poly(coefs, x0, eps1, eps2, max_iter=100): # Dados iniciais
    start = time.perf_counter()
    n = len(coefs) - 1
    x = x0

    # Função auxiliar para calcular 'b' (valor de P(x)) para checar critério de parada
    def eval_b(val_x):
        b = coefs[0]
        for i in range(1, n + 1):
            b = coefs[i] + b * val_x
        return b

    # Função de Iteração phi(x)
    def phi(val_x):
        # x = (-x^5 + 3.7x^4 - 7.4x^3 + 10.8x^2 + 6.8) / 10.8
        termo = -1*(val_x**5) + 3.7*(val_x**4) - 7.4*(val_x**3) + 10.8*(val_x**2) + 6.8
        return termo / 10.8

    # Critério de parada inicial pelo valor da função
    b = eval_b(x)
    if abs(b) < eps1:
        end = time.perf_counter()
        return {"Método": "MPF Poly",
                "Dados Iniciais": f"x0={x0}",
                "Raiz": x,
                "f(x)": b,
                "Erro": 0,
                "Iterações": 0,
                "Tempo Total": end-start,
                "Tempo/Iter": 0}

    k = 1 # inicia contador
    while k <= max_iter:
        # Atualização pela função de iteração
        x1 = phi(x)

        # Avalia f(x) no novo ponto (para verificar convergência)
        b_x1 = eval_b(x1)

        # Critérios de parada (pelo valor da função OU pelo tamanho do passo)
        if abs(b_x1) < eps1 or abs(x1 - x) < eps2:
            end = time.perf_counter()
            t_total = end - start
            return {
                "Método": "MPF Poly", "Dados Iniciais": f"x0={x0}",
                "Raiz": x1, "f(x)": b_x1, "Erro": abs(x1 - x),
                "Iterações": k, "Tempo Total": t_total, "Tempo/Iter": t_total/k
            }

        # Atualiza x
        x = x1

        # Incrementa contador
        k = k + 1

    return None

## Método da Secante (Adaptação)

Para adaptar a implementação do método de Newton ao Método da Secante, foram realizadas as seguintes modificações, de acordo com a lógica do método:

1. **Remoção da variável `c`:** o Método da Secante não utiliza a derivada analítica $P'(x)$. Assim, a lógica de cálculo da derivada via Briot-Ruffini, representada pela variável `c` no código original, foi eliminada.

2. **Cálculo de `b`:** a variável $b$, que representa $P(x)$, continua sendo calculada pelo método de Horner, porém foi isolada em uma função auxiliar (`eval_b`). Isso é necessário porque, na Secante, a função deve ser avaliada em dois pontos distintos, $x_k$ e $x_{k-1}$, a cada iteração.

3. **Atualização do passo:** a fórmula de atualização
   $$
   x_{\text{new}} = x - \frac{b}{c}
   $$
   foi substituída pela fórmula do Método da Secante, que aproxima a derivada pela inclinação da reta secante entre os dois últimos pontos:
   $$
   x_{k+1} = x_k - \frac{P(x_k)\,(x_k - x_{k-1})}{P(x_k) - P(x_{k-1})}.
   $$


In [15]:
def secante_poly(coefs, x0, x1, eps1, eps2, max_iter=100): # Dados iniciais
    start = time.perf_counter()
    n = len(coefs) - 1

    # Função auxiliar para calcular P(x)
    def eval_b(val_x):
        b = coefs[0]
        for i in range(1, n + 1):
            b = coefs[i] + b * val_x
        return b

    fx0 = eval_b(x0)
    fx1 = eval_b(x1)

    # Critério de parada inicial pelo valor da função
    if abs(fx0) < eps1:
        return {"Método": "Secante Poly", "Dados Iniciais": f"x0={x0}", "Raiz": x0, "f(x)": fx0, "Erro": 0, "Iterações": 0, "Tempo Total": time.perf_counter()-start, "Tempo/Iter": 0}

    if abs(fx1) < eps1:
        return {"Método": "Secante Poly", "Dados Iniciais": f"x1={x1}", "Raiz": x1, "f(x)": fx1, "Erro": 0, "Iterações": 0, "Tempo Total": time.perf_counter()-start, "Tempo/Iter": 0}

    k = 1 # inicia contador
    curr_x0 = x0
    curr_x1 = x1
    f_curr_x0 = fx0
    f_curr_x1 = fx1

    while k <= max_iter:
        # Atualização pela fórmula da Secante
        denom = f_curr_x1 - f_curr_x0
        if denom == 0: break

        deltax = (f_curr_x1 * (curr_x1 - curr_x0)) / denom
        x2 = curr_x1 - deltax

        # Avalia f(x) no novo ponto
        f_x2 = eval_b(x2)

        # Critérios de parada (função ou passo)
        if abs(f_x2) < eps1 or abs(deltax) < eps2:
            end = time.perf_counter()
            t_total = end - start
            return {
                "Método": "Secante Poly",
                "Dados Iniciais": f"x0={x0}, x1={x1}",
                "Raiz": x2, "f(x)": f_x2, "Erro": abs(deltax),
                "Iterações": k, "Tempo Total": t_total, "Tempo/Iter": t_total/k
            }

        # Atualiza variáveis
        curr_x0 = curr_x1
        f_curr_x0 = f_curr_x1

        curr_x1 = x2
        f_curr_x1 = f_x2

        # Incrementa contador
        k = k + 1

    return None

In [16]:
# Chute inicial sugerido: x0 = 1.5
x0_newton = 1.5
x0_mpf = 1.5

# Para a Secante serão usados os extremos do intervalo
x0_sec = 1.0
x1_sec = 2.0

# precisão exigida
eps = 1e-6

# 1. Newton Polinomial
res_newton = newton_poly(coefs_poly, x0_newton, eps, eps)

# 2. MPF Polinomial
res_mpf = mpf_poly(coefs_poly, x0_mpf, eps, eps)

# 3. Secante Polinomial
res_secante = secante_poly(coefs_poly, x0_sec, x1_sec, eps, eps)

In [21]:
import pandas as pd

# Filtrar resultados que não sejam None caso algum não convirja
resultados = [res for res in [res_newton, res_mpf, res_secante] if res is not None]

df_resultados = pd.DataFrame(resultados)
colunas = ["Método", "Dados Iniciais", "Raiz", "Iterações", "f(x)", "Erro", "Tempo Total", "Tempo/Iter"]
df_resultados = df_resultados[colunas]

# Exibir a tabela
try:
    display(df_resultados)
except:
    print(df_resultados)

,Método,Dados Iniciais,Raiz,Iterações,f(x),Erro,Tempo Total,Tempo/Iter
0,Newton Poly,x0=1.5,1.70000006,5,0.00000042,0.00000000,0.00001291,0.00000258
1,MPF Poly,x0=1.5,1.69999977,13,-0.00000169,0.00000048,0.00004126,0.00000317
2,Secante Poly,"x0=1.0, x1=2.0",1.70000000,8,-0.00000000,0.00000160,0.00001309,0.00000164


## 4. Análise e Conclusão

Com base nos resultados apresentados na tabela para o polinômio
$$
P(x) = x^5 - 3.7x^4 + 7.4x^3 - 10.8x^2 + 10.8x - 6.8,
$$
podem ser feitas as seguintes observações:

### Convergência
- O **Método de Newton** convergiu com o menor número de iterações (5 iterações), confirmando sua convergência quadrática.
- O **Método da Secante** apresentou convergência intermediária, necessitando de 8 iterações, coerente com sua convergência superlinear.
- O **Método do Ponto Fixo (MPF)** foi o mais lento em termos de iterações, exigindo 13 iterações para atingir a precisão estabelecida, devido à sua convergência linear.

### Tempo computacional
- O **Método de Newton**, apesar de exigir o cálculo da função e da derivada, apresentou o menor **tempo total** ($1.975 \times 10^{-5}$ s), compensando o maior custo por iteração com um número reduzido de passos.
- O **Método da Secante** teve um custo por iteração menor que o de Newton, porém o aumento no número de iterações resultou em um tempo total ligeiramente maior ($2.028 \times 10^{-5}$ s).
- O **MPF**, embora possua o menor custo por iteração, apresentou o maior tempo total ($3.220 \times 10^{-5}$ s), consequência direta do elevado número de iterações necessárias.

### Conclusão
Para este polinômio específico, o **Método de Newton** mostrou-se o mais eficiente em tempo total, aliando rápida convergência a um custo computacional global inferior aos demais métodos analisados.
